# Case 1: Dragless Sphere

The initial atmospheric check-case scenario called for a small sphere to be
dropped from an initial position located 30,000 ft over the Equator/Prime
Meridian intersection. Its initial linear velocity should have matched the
Earth’s eastward rotation rate at that altitude so that it was initially
motionless (in a linear sense) with respect to the still atmosphere; however,
it should have had zero inertial rotation; thus it had a small residual
rotation relative to the Earth below and the atmosphere surrounding it.

The sphere’s orientation was chosen such that the body x-axis was pointed north,
the body z-axis was initially pointed down, with the body y-axis completing the
orthogonal right-hand-rule by pointing east. Thus the vehicle would have had a
small amount of ‘roll rate’ with respect to the rotating Earth. The sphere is
imagined to have no reaction with the atmosphere - no damping, and no drag; it
will accelerate downward toward the ellipsoidal Earth in response to
second-harmonic (J2) gravitation as a function of its geometric height. This is
achieved by setting the coefficient of drag (CD) to zero for this scenario.


 Condition                              | Type
 -------------------------------------- | ------------------------------------
 **Vehicle**                            | Dragless sphere
 **Geodesy**                            | WGS-84 rotating
 **Atmosphere**                         | US 1976 STD; no wind
 **Gravitation**                        | J<sub>2</sub>
 **Duration**                           | 30 s
 **Initial states:**                    | Geodetic, Local-relative Body axes
  &mdash; Position [deg, deg, ft MSL]   | `[0, 0, 30000]`
  &mdash; Velocity [ft/s]               | `[0, 0, 0]`
  &mdash; Attitude [deg]                | `[0, 0, 0]`
  &mdash; Rate [deg/s]                  | `[-0.004178073, 0, 0]`
  **Notes**                             | CD set to zero


# Simulation Results

First look at provided NASA test case:

In [ ]:
from numpy import loadtxt
import matplotlib.pyplot as plt
%matplotlib inline 

columns = loadtxt("test/NASA-testcase_atmos_01.csv", delimiter=',', unpack=True)

n_times              = columns[0]
n_altitudeMsl        = columns[1]
n_longitude          = columns[2]
n_localGravity       = columns[5]
n_altRate            = columns[6]
n_speedSound         = columns[7]

ax = plt.figure(figsize=(16,9))
plt.title(r"NASA Reference Data, Altitude MSL [feet]")
plt.ylabel(r"Altitude MSL [ft]")
plt.xlabel(r"Time [s]")
plt.plot(n_times, n_altitudeMsl)
ax.axes[0].set_ylim([14000,31000])
ax.axes[0].set_xlim([0,30])
plt.show()

Looks like the sphere fell. Great! We'll investigate the other numbers later.


# JSBSim Output

And now we look at our output:

In [ ]:
columns = loadtxt("test/JSBSim-output.csv", delimiter=',', unpack=True, comments="T")
j_times              = columns[ 0]
j_altitudeMsl        = columns[ 1]
j_longitude          = columns[22]
j_localGravity       = columns[32]
j_altRate            = columns[33]*60
j_speedSound         = columns[34]
j_mach               = columns[35]

ax = plt.figure(figsize=(16,9))
plt.title(r"JSBSim Simulation Data, Altitude MSL [feet]")
plt.ylabel(r"Altitude MSL [ft]")
plt.xlabel(r"Time [s]")
plt.plot(n_times, n_altitudeMsl, color='k', lw=6, alpha=0.15, label="NASA Test Cases")
plt.plot(j_times, j_altitudeMsl, label="JSBSim Output")
plt.legend(loc=0)
ax.axes[0].set_ylim([14000,31000])
ax.axes[0].set_xlim([0,30])
plt.show()

So our ball also dropped. Looks like it's working. In all cases, the data should be so close that looking at the raw output won't be useful. Instead we want to look at the difference between the NASA data and our simulation:

# Comparisons

Pointwise diff the two results:

In [ ]:
ax = plt.figure(figsize=(16,7))
plt.title(r"NASA vs JSBSim, Altitude MSL Diff [feet]")
plt.ylabel(r"Altitude MSL Diff [ft]")
plt.xlabel(r"Time [s]")
plt.plot([0,30], [0,0], color='k', alpha=0.2)
plt.plot(j_times, j_altitudeMsl-n_altitudeMsl, color='#709afa')
ax.axes[0].set_xlim([0,30])
plt.show()

print "JSBSim Last Position Diff: %0.5f ft" % (j_altitudeMsl[-1]-n_altitudeMsl[-1])

JSBsim ended up off by about `-0.00013` feet (~`-0.04` mm) over a fall of 14,000 feet!

In [ ]:
ax = plt.figure(figsize=(16,7))
plt.title(r"NASA vs JSBSim, Longitude Diff [deg]")
plt.ylabel(r"Longitude Diff [deg]")
plt.xlabel(r"Time [s]")
plt.plot([0,30], [0,0], color='k', alpha=0.2)
plt.plot(j_times, j_longitude-n_longitude, color='#709afa')
ax.axes[0].set_xlim([0,30])
ax.axes[0].set_ylim([-1e-7,1e-7])
plt.show()

Looks like numerical noise (IEEE float round error). Nothing to see here...

In [ ]:
ax = plt.figure(figsize=(16,7))
plt.title(r"NASA vs JSBSim, Gravity Diff [ft/s/s]")
plt.ylabel(r"Gravity Diff [ft/s/s]")
plt.xlabel(r"Time [s]")
plt.plot([0,30], [0,0], color='k', alpha=0.2)
plt.plot(j_times, j_localGravity-n_localGravity, color='#709afa')
ax.axes[0].set_xlim([0,30])
plt.show()

There was a fair amount of divergence in the sim tools for this value. It didn't seem to bother the authors at this scale.

In [ ]:
ax = plt.figure(figsize=(16,7))
plt.title(r"NASA vs JSBSim, Altitude Rate Diff [ft/min]")
plt.ylabel(r"Altitude Rate Diff [ft/min]")
plt.xlabel(r"Time [s]")
plt.plot([0,30], [0,0], color='k', alpha=0.2)
plt.plot(j_times, j_altRate-n_altRate, color='#709afa')
ax.axes[0].set_xlim([0,30])
plt.show()

In [ ]:
ax = plt.figure(figsize=(16,7))
plt.title(r"NASA vs JSBSim, Speed of Sound Diff [ft/s]")
plt.ylabel(r"Speed of Sound Diff [ft/s]")
plt.xlabel(r"Time [s]")
plt.plot([0,30], [0,0], color='k', alpha=0.2)
plt.plot(j_times, j_speedSound-n_speedSound, color='#709afa')
ax.axes[0].set_xlim([0,30])
ax.axes[0].set_ylim([-1e-2,1e-2])
plt.show()